### Generate an environment
Currently, this just copies data/default_env.pickle into data/env.pickle.

**To do**: Add a DVC parameter such as `env_name` and if this isn't set to "default" then generate a new environment.

In [ ]:
!dvc run --force --always-changed -n gen_env -w .. -d data/default_env.pickle -o data/env.pickle python scripts/gen_env.py

### Generate observations
Currently this checks the values of the parameters `target_area`, `true_norm`, `num_observations` and `obs_data_set`. If they are equal to the settings of the observations in data/default_observations.pickle (and `obs_data_set` has value 1), then that file is copied into data/observations.pickle.

**To do**: Generate a new set of observations if the parameter settings don't match the default one, or `obs_data_set` is not 1.


In [2]:
!dvc run --force --always-changed -n gen_observations -w .. -d data/env.pickle -p target_area,true_norm,num_observations,obs_data_set -o data/observations.pickle python scripts/gen_observations.py

Running stage &#39;gen_observations&#39; with command:
	python scripts/gen_observations.py
(-0.8, 0.7)
(0.25, 0.99)
[3, 8, 13, 35, 40]

ERROR: c:\Users\crast78p\OneDrive - University of Otago\PythonProjects\my fork - learning_norms_with_mcmc_from_pcfg\data\observations.pickle


### Check log likelihood of observation set given no norm and true norm

In [3]:
!dvc run --force --always-changed -n test_obs_log_likelihood1 -w .. -d data/observations.pickle -d data/env.pickle -p true_norm.exp python scripts/test_obs_likelihood.py none true

Running stage &#39;test_obs_log_likelihood1&#39; with command:
	python scripts/test_obs_likelihood.py none true
Log likelihood of observations given no norm: -13732.653608350562

Log likelihood of observations given true exp.: -12940.06839915186

Modifying stage &#39;test_obs_log_likelihood1&#39; in &#39;dvc.yaml&#39;
Updating lock file &#39;dvc.lock&#39;

To track the changes with git, run:

	git add dvc.lock dvc.yaml


### Generate MCMC chains
Creates data/chains_and_log_posteriors.pickle. Also metrics/starts_info.txt, but that is not currently recorded in the "dvc run" command below.

In [4]:
!dvc run --force -n gen_mcmc_chains -w .. -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o data/chains_and_log_posteriors.pickle python scripts/gen_mcmc_chains.py

 Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-13732.653608350562
log(factor) = -0.5108256237659907
log(p_accept)=-164.7561252
E_new rejected
Old Expression: Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-inf
log(factor) = -0.5108256237659907
log(p_accept)=-inf
E_new rejected
Old Expression: Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-13567.861765050382
log(factor) = 0
log(p_accept)=0.0000000
E_new accepted
Old Expression: Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-13622.792379483775
log(factor) = 0
log(p_accept)=-54.9306144
E_new rejected
Old Expression: Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-inf
log(factor) = -0.5108256237659907
log(p_accept)=-inf
E_new rejected
Old Expression: Log-Likelihood=-13567.861765050382
New Expression: Log-Likelihood=-13622.792379483775
log(factor) = 0
log(p_accept)=-54.9306144
E_new rejected
Old Expression: Log-Likelihood=-13567.861765050382
N

### Analyse the chains
Generate file metrics/chain_posteriors.csv and append more information to metrics/chain_info.txt (note: DVC may not handle changing a file very well - perhaps a separate file should be created).

In [5]:
!dvc run --force -n analyse_chains -w .. -d data/chains_and_log_posteriors.pickle -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o metrics/chain_info.txt --plots metrics/chain_posteriors.csv python scripts/analyse_chains.py

70.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.798808747862, -13670.7988087478

### Generate a plot showing norms ordered by likelihood (or now posterior prob.?)
The file plots.html is generated in the root folder, based on data in metrics/chain_posteriors.csv

In [10]:
!cd ..; dvc plots show -t plots/norm_exp_histogram.json metrics/chain_posteriors.csv

The system cannot find the path specified.


### Perform a convergence test to compute $\hat{R}$
The result is written to metrics/conv_test.txt. Also, the posterior sample (after discarding initial warm-up segments of chains and combining the remains) is writen to data/posterior.pickle. 


In [ ]:
!dvc run --force -n conv_test -w .. -d data/chains_and_log_posteriors.pickle -p rhat_step_size -o data/posterior.pickle -o metrics/conv_test.txt python scripts/conv_test.py

### Extract the top norms and compute precision and recall
The results are written to metrics/precision_recall.txt.


In [ ]:
!dvc run --force -n extract_top_norms -w .. -d data/posterior.pickle -d data/env.pickle -p colour_specific,shape_specific,target_area,true_norm -o data/top_norms.pickle -o metrics/precision_recall.txt python scripts/extract_top_norms.py

### Check log likelihood of observation set given top norm

In [ ]:
!dvc run --force --always-changed -n test_obs_log_likelihood2 -w .. -d data/observations.pickle -d data/env.pickle -d data/top_norms.pickle python scripts/test_obs_likelihood.py top